# Importing Libraries

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout,BatchNormalization,Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img,img_to_array

In [ ]:
train_directory='../input/100-bird-species/train'
val_directory='../input/100-bird-species/valid'
test_directory='../input/100-bird-species/test'

In [ ]:
train_datagen=ImageDataGenerator(rescale=1/255)
val_datagen=ImageDataGenerator(rescale=1/255)
test_datagen=ImageDataGenerator(rescale=1/255)

In [ ]:
train_generator=train_datagen.flow_from_directory(train_directory,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 class_mode='sparse',batch_size=256)

val_generator=val_datagen.flow_from_directory(val_directory,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 class_mode='sparse',batch_size=256)

test_gemerator=test_datagen.flow_from_directory(test_directory,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 class_mode='sparse',batch_size=256)

# Classes of Birds 

In [ ]:
bird_dict = train_generator.class_indices

In [ ]:
inv_bird_dict = {v: k.lower().capitalize() for k, v in bird_dict.items()}
inv_bird_dict

# Transfer Learning Using RESNET_101

In [ ]:
from keras.applications import ResNet101V2
convlayer=ResNet101V2(input_shape=(224,224,3),weights='imagenet',include_top=False)
for layer in convlayer.layers:
    layer.trainable=False

In [ ]:
model=Sequential()
model.add(convlayer)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(230,activation='softmax'))
print(model.summary())

In [ ]:
opt=tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer=opt)

In [ ]:
import os
checkpoint_path = "./training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint = ModelCheckpoint(filepath=checkpoint_path, #'./' + 'TRAINING_{epoch:02d}/KAGGLE_{epoch:02d}_{val_loss:.2f}_acc:{val_accuracy:.2f}',
                             save_weights_only=True,
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True)

history=model.fit(train_generator,validation_data=val_generator,
         epochs=5, callbacks=[checkpoint])

### Training with lower learning rate for last iteration

In [ ]:
opt=tf.keras.optimizers.RMSprop(lr=0.0001)
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer=opt)
history_1=model.fit(train_generator,validation_data=val_generator,
         epochs=1, callbacks=[checkpoint])

# Accuracy Plot

In [ ]:
plt.plot(history.history['accuracy'],c='red')
plt.plot(history.history['val_accuracy'],c='green')
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train','validation'],loc='lower right')

# Loss Plot

In [ ]:
plt.plot(history.history['loss'],c='red')
plt.plot(history.history['val_loss'],c='green')
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train','validation'],loc='upper right')

# Test Accuracy 

In [ ]:
model.evaluate(test_gemerator)

# Prediction

Function for predicting bird class

In [ ]:
dic=train_generator.class_indices
icd={k:v for v,k in dic.items()}
def output(location):
    img=load_img(location,target_size=(224,224,3))
    img=img_to_array(img)
    img=img/255
    img=np.expand_dims(img,[0])
    answer=model.predict_classes(img)
    probability=round(np.max(model.predict_proba(img)*100),2)
    #print ('Bird Is',icd[answer[0]], 'With probability',probability)
    print (probability, ' % chances are there that the Bird Is',icd[answer[0]])

### Belted Kingfisher

In [ ]:
img='../input/anurag-mishra/belted.jpg'
pic=load_img('../input/anurag-mishra/belted.jpg',target_size=(224,224,3))
plt.imshow(pic)
output(img)

### ARARIPE MANAKIN 

In [ ]:
img='../input/fdjn-vfvjkfd-v/hg.jpg'
pic=load_img('../input/fdjn-vfvjkfd-v/hg.jpg',target_size=(224,224,3))
plt.imshow(pic)
output(img)

### BANANAQUIT

In [ ]:
img='../input/fj-fdjvdk-vmdvdjl/bananan.jpg'
pic=load_img('../input/fj-fdjvdk-vmdvdjl/bananan.jpg',target_size=(224,224,3))
plt.imshow(pic)
output(img)

In [ ]:
import os
os.chdir(r'/kaggle/working')

In [ ]:
<a href="./KAGGLE_01_0.16_acc:0.95.hdf5"> Download File</a>

<a href="./KAGGLE_01_0.16_acc:0.95.hdf5"> Download File</a>

In [ ]:
import tensorflow as tf
import os
from tensorflow.keras.layers import Input, Dense, Conv2D, Dropout
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import concatenate, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.models import Sequential


convlayer=ResNet101V2(input_shape=(224,224,3),weights='imagenet',include_top=False)
model_resnet=Sequential()
model_resnet.add(convlayer)
model_resnet.add(Dropout(0.5))
model_resnet.add(Flatten())
model_resnet.add(BatchNormalization())
model_resnet.add(Dense(2048,kernel_initializer='he_uniform'))
model_resnet.add(BatchNormalization())
model_resnet.add(Activation('relu'))
model_resnet.add(Dropout(0.5))
model_resnet.add(Dense(1024,kernel_initializer='he_uniform'))
model_resnet.add(BatchNormalization())
model_resnet.add(Activation('relu'))
model_resnet.add(Dropout(0.5))
model_resnet.add(Dense(230,activation='softmax'))
opt=tf.keras.optimizers.Adam(lr=0.001)
model_resnet.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer=opt)
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
model_resnet.load_weights(latest)

In [ ]:
model_resnet.evaluate(test_gemerator)

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

In [ ]:
os.listdir('./training_1')

In [ ]:
!zip -zcvf outputname.zip ./training_1